In [3]:
import numpy as np
import os

In [4]:
def load_data(filename):
    """Load a given txt file.

    Args:
        filename: A string.

    Returns:
        raw_data: An array of shape [n_samples, 256].
        labels : An array of shape [n_samples,].
        
    """
    data= np.load(filename)
    x= data['x']
    y= data['y']
    return x, y

In [5]:
data_dir = "data/"
train_filename = "training.npz"

In [6]:
raw_data, labels = load_data(os.path.join(data_dir, train_filename))

In [12]:
def prepare_X(raw_X):
    """Extract features from raw_X as required.

    Args:
        raw_X: An array of shape [n_samples, 256].

    Returns:
        X: An array of shape [n_samples, n_features].
    """
    raw_image = raw_X.reshape((-1, 16, 16))

	# Feature 1: Measure of Symmetry
	### YOUR CODE HERE
    symmetry  = [-np.sum(abs(image - np.fliplr(image)))/256 for image in raw_image]

	### END YOUR CODE

	# Feature 2: Measure of Intensity
	### YOUR CODE HERE
    intensity = [np.sum(image)/256 for image in raw_image]
	### END YOUR CODE

	# Feature 3: Bias Term. Always 1.
	### YOUR CODE HERE
    bias = [1 for image in raw_image]
	### END YOUR CODE

	# Stack features together in the following order.
	# [Feature 3, Feature 1, Feature 2]
	### YOUR CODE HERE
    X = list(zip(bias,symmetry,intensity))
	### END YOUR CODE
    return X

In [13]:
prepare_X(raw_data)

[(1, -0.1174375, -0.7539140625),
 (1, -0.4378671875, -0.42091796875),
 (1, -0.162296875, -0.77228125),
 (1, -0.24964062499999998, -0.336984375),
 (1, -0.059500000000000004, -0.7692578125),
 (1, -0.05090625000000001, -0.7954375),
 (1, -0.5068046875, -0.17038671875),
 (1, -0.34746875, -0.7159140625),
 (1, -0.16164062500000004, -0.7020703125),
 (1, -0.0686171875, -0.74220703125),
 (1, -0.6294140625, -0.39534765625),
 (1, -0.025328125000000003, -0.712375),
 (1, -0.1780859375, -0.44002734374999997),
 (1, -0.31722656250000003, -0.6714960937500001),
 (1, -0.1006875, -0.646390625),
 (1, -0.5185234375, -0.53352734375),
 (1, -0.6129609375, -0.17041796875000004),
 (1, -0.441828125, -0.465203125),
 (1, -0.264640625, -0.3118046875),
 (1, -0.1967890625, -0.41716796874999995),
 (1, -0.47535156249999994, -0.53076171875),
 (1, -0.5563984375, -0.43349609375),
 (1, -0.36103124999999997, 0.15801562500000002),
 (1, -0.12106250000000002, -0.6343203125),
 (1, -0.3790859375, -0.33129296874999997),
 (1, -0.372